# Future climate uncertainty

In [ ]:
# spatial libraries
import shapely.geometry
import rioxarray as rioxr
import geopandas as gpd
import xarray as xr
import xesmf as xe
import regionmask

from xclim.indicators import atmos
from xclim import core 

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import cftime
import os

os.chdir('/home/rooda/Dropbox/Patagonia/')

## Grid reference

In [ ]:
# Catchment shapefiles
basins = gpd.read_file("GIS South/Basins_Patagonia_ice.shp")
basins = basins.set_index("ID")

# Glacier shapefiles
rgi6 = gpd.read_file("GIS South/Glaciers/RGI6_v2.shp")[["geometry"]]
rgi7 = gpd.read_file("GIS South/Glaciers/RGI7_v2.shp")[["geometry"]]
glaciers  = pd.concat([rgi6.geometry, rgi7.geometry])
glaciers  = glaciers.buffer(0.05) # mask to use for baseline climate

In [ ]:
# high resolution grid
raster_hr   = xr.open_dataset("/home/rooda/OGGM_results/PMET_OGGM_1980_2019m.nc").prcp

# DEMs to downscale temperature
dem_005 = xr.open_dataset("/home/rooda/OGGM_results/PMET_OGGM_1980_2019m.nc").hgt

dem_010 = xr.open_dataset("/home/rooda/OGGM_results/MSWEP_OGGM_1980_2019m.nc").hgt
regridder  = xe.Regridder(dem_010,   dem_005, "nearest_s2d")
dem_010    = regridder(dem_010)

dem_025 = xr.open_dataset("/home/rooda/OGGM_results/ERA5_OGGM_1980_2019m.nc").hgt
regridder  = xe.Regridder(dem_025,   dem_005, "nearest_s2d")
dem_025    = regridder(dem_025)

# lapse rate
lapse_rate = 0.0065 

## Preprocessing bias corrected data

In [ ]:
baseline_period = slice("1980-01-01", "2010-01-01")
future_period   = slice("2070-01-01", "2100-01-01")

lat_coords = np.arange(-56,-40, 0.05)
lon_coords = np.arange(-76,-67, 0.05)

climate_list  = ["PMET", "ERA5", "CR2MET", "MSWEP"]
gcm_list      = ["ACCESS-CM2", "BCC-CSM2-MR", "CMCC-ESM2", "FGOALS-f3-L", "GFDL-ESM4", "CMCC-CM2-SR5", "KACE-1-0-G", "MPI-ESM1-2-HR", "MRI-ESM2-0", "MIROC6"]
ssp_list      = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
bcm_list      = ['MVA', 'DQM', 'MBC']

In [ ]:
os.chdir("/home/rooda/OGGM_results/Future_climate_bc/")

dataset_pp   = []
dataset_prsn = []
dataset_ppd  = []
dataset_t2m  = []

for climate in tqdm(climate_list):
    
    dataset_pp_climate   = []
    dataset_prsn_climate = []
    dataset_ppd_climate  = []
    dataset_t2m_climate  = []
    
    for gcm in tqdm(gcm_list, leave = False):
        for ssp in tqdm(ssp_list, leave = False):
            for bcm in bcm_list: 
        
                # pp and t2m in target resolution
                pp_projection = xr.open_dataset("PP_" + climate + "_" + gcm + "_" + ssp + "_"  + bcm + ".nc").pr
                pp_projection = pp_projection.sel(time = future_period)
                regridder     = xe.Regridder(pp_projection,   raster_hr, "nearest_s2d")
                pp_projection = regridder(pp_projection) ## ESMF.RegridMethod.NEAREST_STOD
                pp_projection.attrs['units']  = "kg m-2 s-1"
                pp_projection.attrs['standard_name']  = "precipitation_flux"
                
                t2m_projection = xr.open_dataset("T2M_" + climate + "_" + gcm + "_" + ssp + "_"  + bcm + ".nc").tas
                t2m_projection = t2m_projection.sel(time = future_period)
                t2m_projection = core.units.convert_units_to(t2m_projection, target = 'C')
                
                if climate == "ERA5": # uses lapse rate
                    regridder  = xe.Regridder(t2m_projection, raster_hr, "nearest_s2d")
                    t2m_projection   = regridder(t2m_projection) 
                    factor     = (dem_025 - dem_005)*lapse_rate
                    t2m_projection   =  t2m_projection + factor
                    
                if climate == "MSWEP": # uses lapse rate
                    regridder  = xe.Regridder(t2m_projection,   raster_hr, "nearest_s2d")
                    t2m_projection  = regridder(t2m_projection)
                    factor     = (dem_010 - dem_005)*lapse_rate
                    t2m_projection  =  t2m_projection + factor
                    
                if climate == "CR2MET" or climate == "PMET":  # same res, different grid
                    regridder  = xe.Regridder(t2m_projection, raster_hr, "nearest_s2d")
                    t2m_projection = regridder(t2m_projection) # simple case (same resolution)
                
                t2m_projection.attrs['units']  = "C"
                t2m_projection.attrs['standard_name']  = "air_temperature"
                
                # calculation of ppd (-1ºC threshold)
                ppd_projection = t2m_projection.where(t2m_projection >= -1)
                ppd_projection = (ppd_projection + 1).resample(time = "YS").sum(skipna = False)
                ppd_projection = ppd_projection.where(ppd_projection > 0)
                ppd_projection = ppd_projection.mean(dim="time")
                ppd_projection = ppd_projection.assign_coords({"CLIMATE": climate, "GCM": gcm, "SSP": ssp, "BCM": bcm})
                ppd_projection = ppd_projection.expand_dims(['CLIMATE', 'GCM', 'SSP', 'BCM'])
                dataset_ppd_climate.append(ppd_projection)    
                
                # calculation of prsn (brown method)
                prsn_projection = atmos.snowfall_approximation(pp_projection, t2m_projection, method='brown', thresh='0 degC')
                prsn_projection = core.units.convert_units_to(prsn_projection, target = 'mm month-1', context = "hydro")
                prsn_projection = prsn_projection.resample(time = "YS").sum(skipna = False)
                prsn_projection = prsn_projection.mean(dim="time")
                prsn_projection = prsn_projection.assign_coords({"CLIMATE": climate, "GCM": gcm, "SSP": ssp, "BCM": bcm})
                prsn_projection = prsn_projection.expand_dims(['CLIMATE', 'GCM', 'SSP', 'BCM'])
                dataset_prsn_climate.append(prsn_projection) 
                
                # calculation of air temperature
                t2m_projection = t2m_projection.resample(time = "YS").mean(skipna = False)
                t2m_projection = t2m_projection.mean(dim="time")
                t2m_projection = t2m_projection.assign_coords({"CLIMATE": climate, "GCM": gcm, "SSP": ssp, "BCM": bcm})
                t2m_projection = t2m_projection.expand_dims(['CLIMATE', 'GCM', 'SSP', 'BCM'])
                dataset_t2m_climate.append(t2m_projection)               
                
                # calculation of total precipitation
                pp_projection = core.units.convert_units_to(pp_projection, target = 'mm month-1', context = "hydro")
                pp_projection = pp_projection.resample(time = "YS").sum(skipna = False)
                pp_projection = pp_projection.mean(dim="time")
                pp_projection = pp_projection.assign_coords({"CLIMATE": climate, "GCM": gcm, "SSP": ssp, "BCM": bcm})
                pp_projection = pp_projection.expand_dims(['CLIMATE', 'GCM', 'SSP', 'BCM'])
                dataset_pp_climate.append(pp_projection)
            
    # trick due to RAM limitation
    dataset_pp_climate = xr.combine_by_coords(dataset_pp_climate).load()
    dataset_pp.append(dataset_pp_climate)
                
    dataset_t2m_climate = xr.combine_by_coords(dataset_t2m_climate).load()
    dataset_t2m.append(dataset_t2m_climate)

    dataset_ppd_climate = xr.combine_by_coords(dataset_ppd_climate).load()
    dataset_ppd.append(dataset_ppd_climate)
                
    dataset_prsn_climate = xr.combine_by_coords(dataset_prsn_climate).load()
    dataset_prsn.append(dataset_prsn_climate)

dataset_pp   = xr.combine_by_coords(dataset_pp)
dataset_t2m  = xr.combine_by_coords(dataset_t2m)
dataset_ppd  = xr.combine_by_coords(dataset_ppd)
dataset_prsn = xr.combine_by_coords(dataset_prsn)

In [ ]:
# mask: only glaciarated area
mask    = regionmask.mask_geopandas(glaciers, dataset_pp)   >= 0

dataset_pp    = dataset_pp.where(mask, drop = True)
dataset_t2m   = dataset_t2m.where(mask, drop = True)
dataset_prsn  = dataset_prsn.where(mask, drop = True).prsn
dataset_ppd   = dataset_ppd.where(mask, drop = True) * 30 # daily

In [ ]:
# summarize arrays

## all
dataset_pp_ensemble_mean  = dataset_pp.mean(dim = ["CLIMATE","GCM", "SSP", "BCM"])
dataset_pp_ensemble_std   = dataset_pp.std(dim = ["CLIMATE","GCM", "SSP", "BCM"])
dataset_t2m_ensemble_mean = dataset_t2m.mean(dim = ["CLIMATE","GCM", "SSP", "BCM"])
dataset_t2m_ensemble_std  = dataset_t2m.std(dim = ["CLIMATE","GCM", "SSP", "BCM"])

## only climate
dataset_pp_climate    = dataset_pp.std(dim = ["CLIMATE"]).mean(dim = ["GCM", "SSP", "BCM"])
dataset_t2m_climate   = dataset_t2m.std(dim = ["CLIMATE"]).mean(dim = ["GCM", "SSP", "BCM"])
dataset_prsn_climate  = dataset_prsn.std(dim = ["CLIMATE"]).mean(dim = ["GCM", "SSP", "BCM"])
dataset_ppd_climate   = dataset_ppd.std(dim = ["CLIMATE"]).mean(dim = ["GCM", "SSP", "BCM"])

## only gcm
dataset_pp_gcm    = dataset_pp.std(dim = ["GCM"]).mean(dim = ["CLIMATE", "SSP", "BCM"])
dataset_t2m_gcm   = dataset_t2m.std(dim = ["GCM"]).mean(dim = ["CLIMATE", "SSP", "BCM"])
dataset_prsn_gcm  = dataset_prsn.std(dim = ["GCM"]).mean(dim = ["CLIMATE", "SSP", "BCM"])
dataset_ppd_gcm   = dataset_ppd.std(dim = ["GCM"]).mean(dim = ["CLIMATE", "SSP", "BCM"])

## only ssp
dataset_pp_ssp    = dataset_pp.std(dim = ["SSP"]).mean(dim = ["GCM", "CLIMATE", "BCM"])
dataset_t2m_ssp   = dataset_t2m.std(dim = ["SSP"]).mean(dim = ["GCM", "CLIMATE", "BCM"])
dataset_prsn_ssp  = dataset_prsn.std(dim = ["SSP"]).mean(dim = ["GCM", "CLIMATE", "BCM"])
dataset_ppd_ssp   = dataset_ppd.std(dim = ["SSP"]).mean(dim = ["GCM", "CLIMATE", "BCM"])

## only bcm
dataset_pp_bcm    = dataset_pp.std(dim = ["BCM"]).mean(dim = ["GCM", "SSP", "CLIMATE"])
dataset_t2m_bcm   = dataset_t2m.std(dim = ["BCM"]).mean(dim = ["GCM", "SSP", "CLIMATE"])
dataset_prsn_bcm  = dataset_prsn.std(dim = ["BCM"]).mean(dim = ["GCM", "SSP", "CLIMATE"])
dataset_ppd_bcm   = dataset_ppd.std(dim = ["BCM"]).mean(dim = ["GCM", "SSP", "CLIMATE"])

In [ ]:
# mean value for each catchment
averager = xe.SpatialAverager(dataset_pp,   basins.geometry, geom_dim_name="avg")

basins["PP_ensemble_mean"]  = averager(dataset_pp_ensemble_mean,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_ensemble_std"]   = averager(dataset_pp_ensemble_std,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_ensemble_mean"] = averager(dataset_t2m_ensemble_mean, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_ensemble_std"]  = averager(dataset_t2m_ensemble_std,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_ensemble_cv"]    = basins["PP_ensemble_std"]  / basins["PP_ensemble_mean"]
basins["T2M_ensemble_cv"]   = basins["T2M_ensemble_std"] / basins["T2M_ensemble_mean"]

basins["PP_climate"]   = averager(dataset_pp_climate,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_climate"]  = averager(dataset_t2m_climate,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PRSN_climate"] = averager(dataset_prsn_climate, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PPD_climate"]  = averager(dataset_ppd_climate,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

basins["PP_gcm"]   = averager(dataset_pp_gcm,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_gcm"]  = averager(dataset_t2m_gcm,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PRSN_gcm"] = averager(dataset_prsn_gcm, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PPD_gcm"]  = averager(dataset_ppd_gcm,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

basins["PP_ssp"]   = averager(dataset_pp_ssp,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_ssp"]  = averager(dataset_t2m_ssp,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PRSN_ssp"] = averager(dataset_prsn_ssp, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PPD_ssp"]  = averager(dataset_ppd_ssp,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

basins["PP_bcm"]   = averager(dataset_pp_bcm,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_bcm"]  = averager(dataset_t2m_bcm,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PRSN_bcm"] = averager(dataset_prsn_bcm, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PPD_bcm"]  = averager(dataset_ppd_bcm,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

In [ ]:
basins = basins[["PP_ensemble_mean", "PP_ensemble_std", 'PP_ensemble_cv', 
                 "T2M_ensemble_mean", "T2M_ensemble_std", 'T2M_ensemble_cv', 
                 'PP_climate','T2M_climate', 'PRSN_climate', 'PPD_climate',
                 'PP_gcm', 'T2M_gcm', 'PRSN_gcm', 'PPD_gcm', 
                 'PP_ssp', 'T2M_ssp', 'PRSN_ssp', 'PPD_ssp', 
                 'PP_bcm', 'T2M_bcm', 'PRSN_bcm', 'PPD_bcm']]

basins.to_csv("/home/rooda/Dropbox/Patagonia/MS2 Results/dataset_climate_future.csv")